In [1]:
# 1- Definir o navegador a ser utilizado para navegar com o Python
# 2- Importar os módulos e bibliotecas
# 3- Entender como funcionam requisições na internet
# 4- Conhecer e mapear o processo de coleta de dados no site do ETF.com
# 5- Achar todos os elementos necessários dentro do HTML do site. Vamos pegar a aba de 'Fund Basics' e a de 'Performance'
# 6- Ler a tabela de dados
# 7- Construir a tabela final

#!pip install webdriver-manager

#!pip install selenium

#!pip install html5lib

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Get: É quando você faz uma pergunta e o servidor respode com uma ação. Ex: Entra num site eo servidor exibe a página
# Post: Quando você por ex faz um login, você posta e o servidor responde baseado num input seu

# Criando o bot que vai acessar (abrir) o Google Chrome
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = 'https://www.etf.com/etfanalytics/etf-finder'

driver.get(url)

# O que vamos mandar o bot fazer?

# 1- Abrir o site
# 2- Mudar a visualização para 100
# 3- Ler a tabela
# 4- Avançar todas as páginas
# 5- Trocar pra outra categoria
# 6- (Voltar para a primeira página) Ler todas as tabelas dessa outra categoria

# Achar todos os elementos do HTML do site
# Quando encontrar o elemento no site, clica com o botão direito no código HTML e 'copy full XPath'

# Se eu executar o próximo código depois de já ter entrado no site, show,
# mas se eu executar tudo ao mesmo tempo, como o Python é mais rápido que o carregamento do JavaScript,
# precisamos usar um time:

time.sleep(5)

# Ele deu um enter no meio do código pra gente conseguir ler, mas quando ele fez isso, excluir o caracter "/"
#antes do 'section' que foi pra baixo, e botou outra barra no final depois das 3 aspas (não entendi pq)

botao_100 = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
                                             section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''') #.click() 

# Com alguns sites bastaria eu dar o '.click()' comentado no código acima, mas no caso desse não funciona
# O '.click()' não serve somente para clicar, mas para outras coisas como colar texto, dar CTRL C, etc... -->
# --> Resumindo, dá pra fazer qualquer ação que a gente faz com o mouse

# Outra forma de clicar segue abaixo:

driver.execute_script('arguments[0].click();', botao_100)

# Número de páginas da tabela. Inspeciono onde está escrito a última página. Ex: 1 de 31 eu inspeciono o 31

numero_paginas = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
                                                  section[2]/div[2]/section[2]/div[2]/div/label[2]''')

#print(numero_paginas.text) # Vai aparecer 'of 31', mas queremos somente a parte numérica

# Fazemos o código abaixo porque o número de páginas pode variar de um dia pro outro:
numero_paginas = numero_paginas.text.replace('of ', '')

# Agora vamos converter essa string '31' (no dia que eu fiz isso, mas é variável) em inteiro
numero_paginas = int(numero_paginas)

# Lendo a tabela através de um loop (Fund Basics que é a primeira aba que queremos ler)

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    

    tabela = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
                                              section[2]/div[2]/div/table''')

    # Agora vamos pegar o conteúdo da tabela. Acima pegamos o objeto 'table', que aqui no Python vira um objeto do Selenium
    html_tabela = tabela.get_attribute('outerHTML')

    # Agora vamos ler o html com o Pandas. Tem que botar o "[0]" no final porque vai vir uma formatação que é correta
    # Não entendi a explicação do pq do "[0]" mas bora
    tabela_final = pd.read_html(html_tabela)[0]

    # Se eu dou o display, pegamos a primeira página do 'Fund Basics'
    #display(tabela_final)
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    # Agora vamos fazer o botão para avançar as páginas
    botao_avancar_pagina = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/
    section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]''')
    
    driver.execute_script('arguments[0].click();', botao_avancar_pagina)

# Juntando as tabelas (cada uma de uma página) da lista em um único tabelão
base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

#display(base_de_dados_completa)

# Lendo a segunda aba agora, a de Performance

# Primeiro vamos apertar nessa nova aba
botao_aba = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
section[2]/div[2]/ul/li[2]/span''')

driver.execute_script('arguments[0].click();', botao_aba)

# Voltando a primeira página
for pagina in range(0, numero_paginas):
    botao_voltar_pagina = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/
    section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[1]''')
    
    driver.execute_script('arguments[0].click();', botao_voltar_pagina)
    
# Agora vamos ler a tabela. Só copiei o que já fizemos duas células acima com a primeira tabela
# Podíamos fazer uma função e só chamá-la novamente. Mas ele acabou não fazendo isso.
# Só mudei o final porque quando muda a aba ainda é a mesma página, logo o mesmo código HTML

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    

    tabela = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
                                              section[2]/div[2]/div/table''')

    # Agora vamos pegar o conteúdo da tabela. Acima pegamos o objeto 'table', que aqui no Python vira um objeto do Selenium
    html_tabela = tabela.get_attribute('outerHTML')

    # Agora vamos ler o html com o Pandas. Tem que botar o "[0]" no final porque vai vir uma formatação que é correta
    # Não entendi a explicação do pq do "[0]" mas bora
    tabela_final = pd.read_html(html_tabela)[0]

    # Se eu dou o display, pegamos a primeira página do 'Fund Basics'
    #display(tabela_final)
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    # Agora vamos fazer o botão para avançar as páginas
    botao_avancar_pagina = driver.find_element('xpath', '''/html/body/div[5]/section/div/div[3]/
    section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]''')
    
    driver.execute_script('arguments[0].click();', botao_avancar_pagina)

# Juntando as tabelas (cada uma de uma página) da lista em um único tabelão
base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

#display(base_de_dados_performance)

# Para fechar o navegador:
driver.quit()

# Juntando as duas tabelas das duas abas (Fund Basics e Performance) em uma tabela máxima

# Setting the index with the first column name ('Ticker') instead of 0, 1, 2, etc...
# O dois códigos abaixo só podem ser executados uma primeira vez. Se executar depois de já
# termos mudado o index, dá erro porque o nome da coluna que o Python procura já está no index

base_de_dados_completa = base_de_dados_completa.set_index('Ticker')
base_de_dados_performance = base_de_dados_performance.set_index('Ticker')

# Agora vamos selecionar apenas algumas colunas da tabela de Performance
#base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

#display(base_de_dados_completa)
#display(base_de_dados_performance)

# Juntando as duas tabelas das duas abas (Fund Basics e Performance) em uma tabela máxima PARTE II

# O Join por default vai pegar a 'coluna' do índice, que tem que ser igual pras duas tabelas, e nesse caso de fato é

#base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)

#base_de_dados_final

In [ ]:
# Análise de dados dos ETF's

#1- Puxar as tabelas
#2- Escolher quais períodos de rentabilidade iremos analisar
#3- Retirar os dados faltantes. Fundos que não possuem períodos necessários ficarão de fora
#4- Transformar as rentabilidades em números decimais, ao invés de string
#5- Juntar as tabelas 
#6- Filtrar os ETFs alavancados
#7- Fazer os rankins de rentabilidade por período e somá-los
#8- Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs

In [ ]:
#1- Puxar as tabelas

# Quando tem um sinal de porcentagem (%) dentro de um DataFrame, significa que estamos lidando com uma string

#base_de_dados_completa
#base_de_dados_performance

In [2]:
#2- Escolher quais períodos de rentabilidade iremos analisar

# No nosso caso, iremos pegar as janelas de 3, 5 e 10 anos


base_de_dados_performance = base_de_dados_performance[['3 Years', '5 Years', '10 Years']]

In [3]:
# 3- Retirar os dados faltantes

base_de_dados_performance = base_de_dados_performance.replace('--', pd.NA)

In [4]:
base_de_dados_performance = base_de_dados_performance.dropna()

In [5]:
# 4- Transformar as rentabilidades en números decimais

base_de_dados_performance['3 Years'] = base_de_dados_performance['3 Years'].str.strip('%').astype(float)/100

base_de_dados_performance['5 Years'] = base_de_dados_performance['5 Years'].str.strip('%').astype(float)/100

base_de_dados_performance['10 Years'] = base_de_dados_performance['10 Years'].str.strip('%').astype(float)/100

,3 Years,5 Years,10 Years
Ticker,,,
SPY,0.0854,0.1119,0.1247
IVV,0.0855,0.1121,0.1252
VTI,0.0809,0.1069,0.1209
VOO,0.0854,0.1122,0.1252
QQQ,0.0959,0.1474,0.1707
...,...,...,...
SZK,-0.2873,-0.2458,-0.2352
LD,0.0609,-0.0556,-0.0253
ERUS,-0.3989,-0.2192,-0.1300


In [6]:
# 5- Juntar as tabelas

base_de_dados_final_dados = base_de_dados_completa.join(base_de_dados_performance, how = 'inner')

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B,0.0854,0.1119,0.1247
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B,0.0855,0.1121,0.1252
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B,0.0809,0.1069,0.1209
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,0.0854,0.1122,0.1252
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B,0.0959,0.1474,0.1707
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,-0.2873,-0.2458,-0.2352
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,0.0609,-0.0556,-0.0253
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.3989,-0.2192,-0.1300


In [7]:
# 6- Filtrar os ETFs alavancados (Que contém a palavra 'Leveraged' na coluna 'Segment')

base_de_dados_final_dados = base_de_dados_final_dados[~base_de_dados_final_dados['Segment'].str.contains('Leveraged')]

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B,0.0854,0.1119,0.1247
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B,0.0855,0.1121,0.1252
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B,0.0809,0.1069,0.1209
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,0.0854,0.1122,0.1252
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B,0.0959,0.1474,0.1707
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.97M,-0.2742,-0.2706,-0.2864
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,-0.2873,-0.2458,-0.2352
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,0.0609,-0.0556,-0.0253


In [8]:
# 7- Fazer os rankings de rentabilidade por período e somá-los

# Vamos criar uma nova coluna com o ranking da coluna de '3 Years' e assim por diante
base_de_dados_final_dados['rank_3_anos'] = base_de_dados_final_dados['3 Years'].rank(ascending = False)

base_de_dados_final_dados['rank_5_anos'] = base_de_dados_final_dados['5 Years'].rank(ascending = False)

base_de_dados_final_dados['rank_10_anos'] = base_de_dados_final_dados['10 Years'].rank(ascending = False)

base_de_dados_final_dados

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B,0.0854,0.1119,0.1247,211.5,98.0,77.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B,0.0855,0.1121,0.1252,208.5,96.5,75.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B,0.0809,0.1069,0.1209,248.5,119.0,100.0
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,0.0854,0.1122,0.1252,211.5,94.5,75.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B,0.0959,0.1474,0.1707,169.5,20.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.97M,-0.2742,-0.2706,-0.2864,804.0,809.0,818.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,-0.2873,-0.2458,-0.2352,806.0,805.0,807.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,0.0609,-0.0556,-0.0253,358.0,761.0,714.0


In [9]:
base_de_dados_final_dados['rank_final'] = (base_de_dados_final_dados['rank_3_anos'] +
                                     base_de_dados_final_dados['rank_5_anos'] +
                                     base_de_dados_final_dados['rank_10_anos'])

base_de_dados_final_dados

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B,0.0854,0.1119,0.1247,211.5,98.0,77.5,387.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B,0.0855,0.1121,0.1252,208.5,96.5,75.5,380.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B,0.0809,0.1069,0.1209,248.5,119.0,100.0,467.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,0.0854,0.1122,0.1252,211.5,94.5,75.5,381.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B,0.0959,0.1474,0.1707,169.5,20.0,11.0,200.5
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.97M,-0.2742,-0.2706,-0.2864,804.0,809.0,818.0,2431.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.30M,-0.2873,-0.2458,-0.2352,806.0,805.0,807.0,2418.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,0.0609,-0.0556,-0.0253,358.0,761.0,714.0,1833.0


In [10]:
# 8- Ordenar do menor ranking pro maior, encontrando os 'melhores' ETFs

melhores_etfs = base_de_dados_final_dados.sort_values(by = 'rank_final')

melhores_etfs

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.43B,0.2914,0.2789,0.1693,11.0,1.0,12.0,24.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.34B,0.2405,0.2565,0.2418,23.0,2.0,1.0,26.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Renewable Energy,First Trust,0.58%,$1.81B,0.2543,0.2451,0.1843,18.0,3.0,6.5,27.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$7.72B,0.1962,0.2146,0.2289,36.0,4.0,4.0,44.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$569.09M,0.1921,0.1996,0.2345,39.0,6.0,2.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...
FAZ,Direxion Daily Financial Bear 3X Shares,Inverse Equity: U.S. Financials,Direxion,1.01%,$176.59M,-0.5694,-0.4925,-0.4594,832.0,832.0,831.0,2495.0
DUST,Direxion Daily Gold Miners Index Bear 2x Shares,Inverse Equity: Global Gold Miners,Direxion,0.92%,$118.20M,-0.5502,-0.5481,-0.5270,829.0,834.0,835.0,2498.0
SSG,ProShares UltraShort Semiconductors,Inverse Equity: U.S. Semiconductors,ProShares,0.95%,$6.73M,-0.5726,-0.5168,-0.4782,833.0,833.0,833.0,2499.0


In [11]:
melhores_etfs.head(10)

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.43B,0.2914,0.2789,0.1693,11.0,1.0,12.0,24.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.34B,0.2405,0.2565,0.2418,23.0,2.0,1.0,26.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Renewable Energy,First Trust,0.58%,$1.81B,0.2543,0.2451,0.1843,18.0,3.0,6.5,27.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$7.72B,0.1962,0.2146,0.2289,36.0,4.0,4.0,44.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$569.09M,0.1921,0.1996,0.2345,39.0,6.0,2.0,47.0
SOXX,iShares Semiconductor ETF,Equity: U.S. Semiconductors,Blackrock,0.35%,$7.38B,0.1872,0.2104,0.2339,43.0,5.0,3.0,51.0
PTF,Invesco DWA Technology Momentum ETF,Equity: U.S. Information Technology,Invesco,0.60%,$218.89M,0.1399,0.1860,0.1618,73.0,8.0,16.0,97.0
RWJ,Invesco S&P SmallCap 600 Revenue ETF,Equity: U.S. - Small Cap,Invesco,0.39%,$1.18B,0.2529,0.1414,0.1275,19.0,23.0,65.5,107.5
GRID,First Trust NASDAQ Clean Edge Smart Grid Infra...,Equity: Global Infrastructure,First Trust,0.58%,$705.96M,0.1960,0.1554,0.1317,37.0,16.5,55.5,109.0
